In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.13"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.13

In [2]:
#!import ./../config/Settings.cs

using System;
using Azure;
using Azure.AI.OpenAI;
using System.Net.Http;
using Newtonsoft.Json;

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("./../config/settings.json");

string endpoint = azureEndpoint;
string key = apiKey;
model = "gpt-4-vision";

In [6]:
string GPT4V_KEY = key; // Set your key here

// string QUESTION = "Please describe the picture";
// string IMAGE_PATH = "./Lavaredo.jpeg"; // Set your image path here

// string QUESTION = "How many steps are needed to complete the entire process?";
// string IMAGE_PATH = "./flow.png"; // Set your image path here

// string QUESTION = "L'immagine rappresenta una fattura. Quali sono le voci principali della fattura?";
// string IMAGE_PATH = "./fattura.jpg"; // Set your image path here

// string QUESTION = "L'immagine rappresenta una fattura. Quali sono le voci principali della fattura?";
// string IMAGE_PATH = "./fat_1.jpg"; // Set your image path here

string QUESTION = "Please describe the picture";
string IMAGE_PATH = "./landscape.jpg"; // Set your image path here

string GPT4V_ENDPOINT  = azureEndpoint + @"/openai/deployments/" + model + "/chat/completions?api-version=2023-07-01-preview";

var encodedImage = Convert.ToBase64String(File.ReadAllBytes(IMAGE_PATH));
using (var httpClient = new HttpClient())
{
    httpClient.DefaultRequestHeaders.Add("api-key", GPT4V_KEY);
    var payload = new
    {
        messages = new object[]
        {
            new {
                role = "system",
                content = new object[] {
                    new {
                        type = "text",
                        text = "You are an AI assistant that helps people find information."
                    }
                }
            },
            new {
                role = "user",
                content = new object[] {
                    new {
                        type = "image_url",
                        image_url = new {
                            url = $"data:image/jpeg;base64,{encodedImage}"
                        }
                    },
                    new {
                        type = "text",
                        text = QUESTION
                    }
                }
            }
        },
        temperature = 0.7,
        top_p = 0.95,
        max_tokens = 800,
        stream = false
    };

    var response = await httpClient.PostAsync(GPT4V_ENDPOINT , new StringContent(JsonConvert.SerializeObject(payload), Encoding.UTF8, "application/json"));

    if (response.IsSuccessStatusCode)
    {
        var responseData = JsonConvert.DeserializeObject<dynamic>(await response.Content.ReadAsStringAsync());
        //Console.WriteLine(responseData);
        Console.WriteLine(responseData.choices[0].message.content);
    }
    else
    {
        Console.WriteLine($"Error: {response.StatusCode}, {response.ReasonPhrase}");
    }
}

The picture shows a beautiful natural landscape. It's a sunny day with a clear blue sky that has a few wispy clouds scattered throughout. A wooden boardwalk stretches straight ahead through a lush meadow of tall green grass. The grassland is vibrant and appears to be thriving, with a few clusters of shrubbery and trees in the distance. The boardwalk invites one to walk through the field and enjoy the serenity of the natural surroundings. The lighting suggests it could be late afternoon, with the sun casting a warm glow across the scene. Overall, it's a peaceful setting that conveys a sense of tranquility.


In [7]:
// const string rawImageUri = "https://upload.wikimedia.org/wikipedia/commons/8/8c/Drei_Zinnen-Tre_Cime_Di_Lavaredo_6.JPG";

var encodedImage = Convert.ToBase64String(File.ReadAllBytes(IMAGE_PATH));
string rawImageUri = $"data:image/jpg;base64,{encodedImage}";

OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

ChatCompletionsOptions chatCompletionsOptions = new()
{
    DeploymentName = model,
    MaxTokens = 800,
    Messages =
    {
        new ChatRequestSystemMessage("You are a helpful assistant that describes images."),
        new ChatRequestUserMessage(
            new ChatMessageTextContentItem("Hi! Please describe this image"),
            new ChatMessageImageContentItem(new Uri(rawImageUri)))
    }
};

Response<ChatCompletions> chatResponse = await client.GetChatCompletionsAsync(chatCompletionsOptions);
ChatChoice choice = chatResponse.Value.Choices[0];
if (choice.FinishDetails is StopFinishDetails stopDetails)
{
    Console.WriteLine($"{choice.Message.Role}: {choice.Message.Content}");
}

Error: System.UriFormatException: Invalid URI: The Uri string is too long.
   at System.Uri.CreateThis(String uri, Boolean dontEscape, UriKind uriKind, UriCreationOptions& creationOptions)
   at System.Uri..ctor(String uriString)
   at Submission#8.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [6]:
choice

Azure.AI.OpenAI.ChatChoice Message Azure.AI.OpenAI.ChatResponseMessage Role assistant Content The image shows the impressive natural rock formations known as the Tre Cime di Lav ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall <null> AzureExtensionsContext <null> Index 0 FinishReason <null> FinishDetails Azure.AI.OpenAI.MaxTokensFinishDetails ContentFilterResults Azure.AI.OpenAI.ContentFilterResultsForChoice Sexual Azure.AI.OpenAI.ContentFilterResult Severity safe Filtered False Violence Azure.AI.OpenAI.ContentFilterResult Severity safe Filtered False Hate Azure.AI.OpenAI.ContentFilterResult Severity safe Filtered False SelfHarm Azure.AI.OpenAI.ContentFilterResult Severity safe Filtered False Profanity <null> CustomBlocklists [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) Error <null> ProtectedMaterialText <null> ProtectedMaterialCode <null> Enhancements <null>